In [1]:
import os
from langchain.chat_models import init_chat_model
from langchain_groq import ChatGroq


/Users/manasvi/Github/Agents-Langchain-Langraph/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
llm = init_chat_model("groq:llama-3.1-8b-instant")

In [4]:
import sqlite3

# 1. Create / connect to SQLite database (file will be created in your system)
connection = sqlite3.connect("my_database.db")   # Creates my_database.db in current folder
cursor = connection.cursor()
cursor.execute("SELECT * FROM users;")
print(cursor.fetchall())

[(1, 'Alice', 'alice@example.com'), (2, 'Bob', 'bob@example.com'), (3, 'Charlie', 'charlie@example.com')]


In [9]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///my_database.db")

print(f"Dialect: {db.dialect}")
print(f"Available tables: {db.get_usable_table_names()}")
print(f'Sample output: {db.run("SELECT * FROM users;")}')

Dialect: sqlite
Available tables: ['orders', 'users']
Sample output: [(1, 'Alice', 'alice@example.com'), (2, 'Bob', 'bob@example.com'), (3, 'Charlie', 'charlie@example.com')]


In [11]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()

for tool in tools:
    print(f"{tool.name}: {tool.description}\n")

sql_db_query: Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.

sql_db_schema: Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3

sql_db_list_tables: Input is an empty string, output is a comma-separated list of tables in the database.

sql_db_query_checker: Use this tool to double check if your query is correct before executing it. Always use this tool before executing a query with sql_db_query!



In [17]:
system_prompt = f"""
You are an agent designed to interact with a SQL database.

The database dialect is {db.dialect}.
The only tables that exist are: {', '.join(db.get_usable_table_names())}.

Strict rules:
1. FIRST call `sql_db_list_tables` ONCE to see the tables.
2. When you call `sql_db_schema`, you MUST pass a comma-separated list made ONLY from
   those real table names (e.g. "users" or "users, orders"). NEVER use placeholders
   like "table1,table2".
3. Then use `sql_db_query_checker` to verify each SQL query.
4. Finally call `sql_db_query` to execute the query and answer the user.

Do NOT invent table names. Do NOT skip steps 1–3.
""".format(
    dialect=db.dialect,
    top_k=2,
)

from langchain.agents import create_agent


agent = create_agent(
    llm,
    tools,
    system_prompt=system_prompt,
)

In [18]:
query = "Return the names of the users"
# agent.invoke({"input": query})

for event in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Return the names of the users
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (53c4xet18)
 Call ID: 53c4xet18
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

orders, users


BadRequestError: Error code: 400 - {'error': {'message': "Failed to call a function. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': '<function=sql_db_schema>{table_names="users,orders"}</function>'}}